<h1>概率图模型采样方法</h1>

> <b>解决问题</b>

> 从数据分布中采样，尤其是高维数据分布

参考文献：

[1] Pattern recognition and machine learning, Chapter 11.

[2] Machine Learning, A Probabilistic Perspective, Chapter 23, 24.

# 1. 标准分布直接采样 Sampling from Standard Distribution

>定理: $F$是我们要采样的函数$f$的累积密度函数(cdf)，如果$U\sim Uniform(0, 1)$， 则$F^{-1}(U)\sim F$.
>证明见MLAPP.

因此，如果需要采样$f$，可以先采样$u\sim U(0, 1)$，再计算$x=F^{-1}(u)$。需要计算累积密度函数的逆函数，如果n




# 2. 拒绝采样 Rejection Sampling
